In [ ]:
!pip uninstall -y telegram python-telegram-bot telebot pyTelegramBotAPI
!pip install python-telegram-bot==20.6
!pip install openai-whisper
!pip install torch torchvision torchaudio
!pip install librosa soundfile


Found existing installation: python-telegram-bot 13.15
Uninstalling python-telegram-bot-13.15:
  Successfully uninstalled python-telegram-bot-13.15
  Using cached python_telegram_bot-20.6-py3-none-any.whl.metadata (15 kB)
Using cached python_telegram_bot-20.6-py3-none-any.whl (548 kB)


In [3]:
!pip install nest_asyncio

In [1]:
import pickle

# Load TF-IDF vectorizer and Logistic Regression model
with open("tfidf_vectorizer (1).pkl", "rb") as f:
    vectorizer = pickle.load(f)

with open("logistic_regression_model.pkl", "rb") as f:
    model = pickle.load(f)

def predict_toxicity(text):
    # Convert text to TF-IDF features
    features = vectorizer.transform([text])
    # Predict using logistic regression
    pred = model.predict(features)[0]
    prob = model.predict_proba(features)[0][1]

    # Return both label and probability
    if pred == 1:
        return f"Toxic 😡 (confidence: {prob:.2f})"
    else:
        return f"Non-toxic 😇 (confidence: {1 - prob:.2f})"

In [2]:
import whisper
from pydub import AudioSegment

# Load Whisper model (choose: tiny, base, small, medium, large)
whisper_model = whisper.load_model("base.en")

def transcribe_voice(ogg_path):
    # Convert OGG to WAV
    audio = AudioSegment.from_ogg(ogg_path)
    wav_path = "converted.wav"
    audio.export(wav_path, format="wav")

    # Transcribe using Whisper
    result = whisper_model.transcribe(wav_path)
    return result["text"].strip()


In [4]:
import nest_asyncio
nest_asyncio.apply()  # Allow nested asyncio loops

from telegram import Update
from telegram.ext import ApplicationBuilder, MessageHandler, ContextTypes, filters
import os, asyncio

TOKEN = "8258212437:AAGyWHHN3NkOzUOx5GTNkAMOV7Lg-h0IAUI"

async def handle_voice(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.message.from_user.first_name
    await update.message.reply_text(f"🎙️ Hey {user}, I got your voice! Transcribing...")

    # Download voice
    file = await context.bot.get_file(update.message.voice.file_id)
    await file.download_to_drive("voice.ogg")

    # Transcribe (replace with your Whisper function)
    text = transcribe_voice("voice.ogg")
    await update.message.reply_text(f"🗣 You said: {text}")

    # Predict toxicity
    result = predict_toxicity(text)
    await update.message.reply_text(f"🧠 Result: {result}")

    os.remove("voice.ogg")

# Build bot
app = ApplicationBuilder().token(TOKEN).build()
app.add_handler(MessageHandler(filters.VOICE, handle_voice))

# Run bot in Colab safely
async def main():
    await app.initialize()
    await app.start()
    await app.updater.start_polling()
    print("🤖 Bot is running...")
    await asyncio.Event().wait()

# Call coroutine directly
await main()


🤖 Bot is running...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


CancelledError: 